## 连接

### 连接的基本概念

连接, 是将两张相关的表中不同数据进行整合的方式. 连接常见的模式有四种: 分为左连接 `left` 、右连接 `right` 、内连接 `inner` 、外连接 `outer`. 连接时有一个很重要的参数是连接的键, 表示连接时参照的数据列, 一般使用 `on` 进行定义.

它们的差别主要体现在连接时选取的基准以及保存数据的标准上.

![](https://s1.vika.cn/space/2022/09/13/a7c4516ebb1e4b24b2ff609eca41423b)

- 左连接以左表的数据为准, 不属于左表的数据则不会出现.
- 右连接以右表的数据为准, 不属于右表的数据则不会出现.
- 内连接则是选取两表中都存在的数据.
- 外连接选取两表里的所有数据, 因此又称为 全连接.

### 值连接: merge

为了实现上图中提到的根据列表数据中的某一列的值来进行连接, 可以使用 `merge` 函数进行操作. 

- 在使用时常见的参数有:

	- on: 参考的列名
	- how: 定义连接的方式
	- left_on/right_on: 在两个表想要合并的参考标准列名不同时
	- suffixes: 在出现了重复的列名但是表示不同意思的时候, 可以使用 suffixes 参数来给两列重命名
	- validate: 检测数据的唯一性模式, 有 1:1, 1:m, m:1 连接模式.

### 索引连接: join

索引连接即为将索引视为列, 这种情况使用 join 函数进行实现.

join 函数具有和 merge 函数相似的两个参数: on, how, 除此之外还有和 suffixes 相似的 lsuffix, rsuffix. 使用这些参数来定义 join 函数可以让其与 merge 函数起到相似的作用.

### 方向连接

#### concat

前文提到过的关系型连接是根据数据的内容和相关性进行连接, 除此之外, 有时候我们单纯地希望两个数据进行横向或者纵向的拼接. 这个时候就可以使用 concat 函数.

默认 axis=0, 为纵向拼接, 1 为横向拼接. 除此之外, concat 的默认连接模式为外连接, 但可以通过 join 参数来调整连接模式.

当确认要使用多表直接的方向合并时, 尤其是横向的合并, 可以先用 `reset_index` 方法恢复默认整数索引再进行合并, 防止出现由索引的误对齐和重复索引的笛卡尔积带来的错误结果.

keys 参数可以在连接后仍然显示数据的来源, 通过多重索引的展示形式将其来源的表格显示出来.

#### append, assign

如果想要把一个序列追加到表的行末或者列末, 则可以分别使用 `append` 和 `assign` 方法.

使用 assign 和直接使用 `df[Name]` 的方式去定义列的区别是, 这样会只返回一个临时脚本, 而不会影响原数据.

### 类连接方法

#### compare

它能够比较两个表或者序列的不同处并将其汇总展示

#### combine

让两张表按照一定的规则进行组合, 在进行规则比较时会自动进行列索引的对齐

## 四、练习
### Ex1：美国疫情数据集

现有美国4月12日至11月16日的疫情报表（在`/data/us_report`文件夹下），请将`New York`的`Confirmed, Deaths, Recovered, Active`合并为一张表，索引为按如下方法生成的日期字符串序列：

In [2]:
import pandas as pd
import numpy as np

date = pd.date_range('20200412', '20201116').to_series()
date = date.dt.month.astype('string').str.zfill(2) +'-'+ date.dt.day.astype('string').str.zfill(2) +'-'+ '2020'
date = date.tolist()
date[:5]

['04-12-2020', '04-13-2020', '04-14-2020', '04-15-2020', '04-16-2020']

In [12]:
res = []
for i in date:
    df = pd.read_csv('./data/us_report/' + i + '.csv', index_col='Province_State')
    res.append(df.loc['New York', ['Confirmed', 'Deaths', 'Recovered', 'Active']].to_frame().T)
    
res1 = pd.concat(res)
res1.index= date
res1

,Confirmed,Deaths,Recovered,Active
04-12-2020,189033,9385,23887.0,179648
04-13-2020,195749,10058,23887.0,185691.0
04-14-2020,203020,10842,23887.0,192178.0
04-15-2020,214454,11617,23887.0,202837.0
04-16-2020,223691,14832,23887.0,208859.0
...,...,...,...,...
11-12-2020,545762,33975,81198.0,430589.0
11-13-2020,551163,33993,81390.0,435780.0
11-14-2020,556551,34010,81585.0,440956.0
11-15-2020,560200,34032,81788.0,444380.0


### Ex2：实现join函数

请实现带有`how`参数的`join`函数

* 假设连接的两表无公共列
* 调用方式为 `join(df1, df2, how="left")`
* 给出测试样例

In [13]:
def join(df1, df2, how='left'):
    res_col = df1.columns.tolist() +  df2.columns.tolist()
    dup = df1.index.unique().intersection(df2.index.unique())
    res_df = pd.DataFrame(columns = res_col)
    for label in dup:
        cartesian = [list(i)+list(j) for i in df1.loc[label].values.reshape(-1,1) for j in df2.loc[label].values.reshape(-1,1)]
        dup_df = pd.DataFrame(cartesian, index = [label]*len(cartesian), columns = res_col)
        res_df = pd.concat([res_df,dup_df])
    if how in ['left', 'outer']:
        for label in df1.index.unique().difference(dup):
            if isinstance(df1.loc[label], pd.DataFrame):
                cat = [list(i)+[np.nan]*df2.shape[1] for i in df1.loc[label].values]
            else:
                cat = [list(i)+[np.nan]*df2.shape[1] for i in df1.loc[label].to_frame().values]
            dup_df = pd.DataFrame(cat, index = [label]*len(cat), columns = res_col)
            res_df = pd.concat([res_df,dup_df])
    if how in ['right', 'outer']:
        for label in df2.index.unique().difference(dup):
            if isinstance(df2.loc[label], pd.DataFrame):
                cat = [[np.nan]+list(i)*df1.shape[1] for i in df2.loc[label].values]
            else:
                cat = [[np.nan]+list(i)*df1.shape[1] for i in df2.loc[label].to_frame().values]
            dup_df = pd.DataFrame(cat, index = [label]*len(cat), columns = res_col)
            res_df = pd.concat([res_df,dup_df])
    return res_df

In [14]:
df1 = pd.DataFrame({'col1':[1,2,3,4,5]}, index=list('AABCD'))
df1

,col1
A,1
A,2
B,3
C,4
D,5
